In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import mode
from sklearn import linear_model
import matplotlib
import matplotlib.pyplot as plt
from sklearn import discriminant_analysis
from sklearn.decomposition import PCA
from sklearn import preprocessing
import json
import re
import ast
from bs4 import BeautifulSoup
import urllib
%matplotlib inline

In [117]:
with open('buildings') as f:
    for i in xrange(10327):
        r = urllib.urlopen('http://streeteasy.com/buildings/nyc?page=%d' % (i + 1)).read()
        soup = BeautifulSoup(r)
        for d in soup.find_all('div', class_='details-title'):
             f.write(d.find('a')['href'] + '\n')
            

In [14]:
import urllib 
from bs4 import BeautifulSoup
import json
import sys

key_fields = set([
    'Highlights',
    'Building Amenities',
    'Building Facts',
    'Units',
    'Nearby',
])

def get_line(text, lineno):
    return text.split('\n', lineno + 1)[lineno].strip()

def read_int(text):
    digits = filter(str.isdigit, str(text))
    return (int(digits) if digits != '' else None)

def get_next_spans(td):
    td.findNext('td').find_all('span')

def rip_details(url):
    content = {}
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r)
    for h2 in soup.find_all('h2'):
        if h2.text == 'Highlights':
            content[h2.text] = [get_line(li.text, 2) for li in h2.parent.find_all('li')]
        elif h2.text == 'Building Amenities':
            content[h2.text] = [li.text for li in h2.parent.find_all('li') if not li.text.strip().startswith('googletag')]
        elif h2.text == 'Building Facts':
            for td in h2.parent.find_all('td'):
                if td.text.strip() == 'Facts':
                    keys = ['Units', 'Stories', 'Built Year']
                    f = dict(zip(keys, [read_int(sp.text) for sp in td.findNext('td').find_all('span')]))
                    content.update(f)
                elif td.text.strip() == 'Building Class':
                    content.update({'Building Class' : get_line(td.findNext('span').text, 1)})
                elif td.text.strip() == 'District':
                    keys = ['Community', 'City Council', 'Police']
                    f = dict(zip(keys, [sp.text for sp in td.findNext('td').find_all('span')]))
                    content.update(f)
                elif td.text.strip() == 'Architect:' or td.text.strip() == 'Developer:' or td.text.strip() == 'Interiors:':
                    content.update({td.text.strip()[:-1]: td.findNext('td').text})
        elif h2.text == 'Nearby':
            for h6 in h2.parent.find_all('h6'):
                if h6.text == 'Transportation':
                    content.update({'Transportation': [([sp.text for sp in p.find_all('span')], p.findNext('b').text) for p in h6.parent.find_all('p')]})
                elif h6.text.startswith('Schools'):
                    content.update({'School District': h6.findNext('b').text})
        content.update(zip(['Street', 'City', 'State', 'Zipcode'], soup.body.find_all('div', class_='main-info')[0].h2.text.split(',')))
    return content

with open('buildings') as f, open('details', 'w') as fw:
    for i, l in enumerate(f):
        url = 'http://streeteasy.com' + l.strip()
        try:
            fw.write(json.dumps(rip_details(url)) + '\n')
        except Exception as e:
            sys.stderr.write('%s: %s' % (url, str(e)))
        if i != 0 and i % 1000:
            print l.strip()
        

/building/65-bank-street-new_york
/building/atelier-condominium
/building/47_38-5-street-long_island_city
/building/150-west-51-street-new_york
/building/avalon-clinton
/building/w-downtown-hotel-residences
/building/75-wall-street-new_york
/building/jw-marriott-essex-house
/building/the-sheffield
/building/orion-condominium
/building/the-corinthian
/building/the-margo
/building/plaza-hotel
/building/the-centurion
/building/234-mott-street-new_york
/building/15-william-street-new_york
/building/20-pine-the-collection
/building/56-leonard-street-new_york
/building/halo-lic
/building/grand-two
/building/the-residences-at-400-fifth-avenue
/building/greenwich-club
/building/ava-dobro
/building/30-park-place-new_york
/building/the-brooklyner
/building/trump-world-tower
/building/the-caledonia
/building/234-mott-street-new_york
/building/172-madison-avenue-new_york
/building/trump-place-120
/building/manhattan-house
/building/trump-tower
/building/the-aldyn
/building/manhattan-place
/buildin

KeyboardInterrupt: 